<a href="https://colab.research.google.com/github/park-hoyeon/park-hoyeon.github.io/blob/master/skt_7_08_Haar_Cascade%EC%96%BC%EA%B5%B4%EC%9D%B8%EC%8B%9D_%EC%97%B0%EA%B2%B0%EC%9A%94%EC%86%8C%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python numpy matplotlib ffmpeg-python --quiet
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import ffmpeg
import os
from IPython.display import HTML, Video
print("OpenCV version:", cv.__version__)


In [ ]:
!wget -q https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml -O haarcascade_frontalface_default.xml
if not os.path.exists('haarcascade_frontalface_default.xml'):
    print("Cascade XML download failed!")
else:
    print("Cascade XML downloaded successfully.")

In [ ]:
!wget -q https://raw.githubusercontent.com/opencv/opencv/master/samples/data/lena.jpg -O lena.jpg
if not os.path.exists('lena.jpg'):
    print("lena.jpg download failed. Please upload an image.")
else:
    print("lena.jpg downloaded.")

In [ ]:
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
# 테스트 이미지 불러오기
img_path = 'lena.jpg'  # 혹은 본인이 업로드한 이미지 경로
img = cv.imread(img_path)
if img is None:
    raise FileNotFoundError("Image not found: "+img_path)
# 그레이스케일 변환
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# 얼굴 검출
faces = face_cascade.detectMultiScale(
    gray,
    scaleFactor=1.1,  # 이미지 피라미드 스케일
    minNeighbors=5,   # 얼굴로 판정하기 위한 최소 이웃
    minSize=(30, 30)  # 얼굴로 볼 수 있는 최소 크기
)
print(f"Detected faces: {len(faces)}")
# 얼굴 위치에 사각형 그리기
for (x, y, w, h) in faces:
    cv.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
# Matplotlib으로 시각화
plt.figure(figsize=(6,6))
img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.imshow(img_rgb)
plt.title('Face Detection Result')
plt.axis('off')
plt.show()

# 4. 동영상/웹캠 기반 얼굴 인식


In [ ]:
## 예시 영상 다운로드 (얼굴이 있는 영상 샘플)
# OpenCV 4.x 버전 저장소에 face-detection-video.webm 샘플이 있으므로 이를 사용.
!wget -q https://videos.pexels.com/video-files/5125919/5125919-sd_960_506_30fps.mp4 -O face_sample.mp4
if not os.path.exists('face_sample.mp4'):
    print("Sample video download failed!")
else:
    print("Sample video downloaded.")


In [ ]:
cap = cv.VideoCapture('face_sample.mp4')
#cap = cv.VideoCapture('james.mp4')
if not cap.isOpened():
    print("Cannot open video!")
else:
    # 동영상을 저장할 VideoWriter 설정
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    width  = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv.CAP_PROP_FPS)
    if fps <= 0:
        fps = 25  # 혹시 0으로 읽히면 임의로 25로 설정
    out = cv.VideoWriter('face_detect_out.avi', fourcc, fps, (width, height))
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )
        for (x, y, w, h) in faces:
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        out.write(frame)  # 결과 프레임을 파일로 저장
    cap.release()
    out.release()
    print("Video face detection done. Saved as face_detect_out.avi")
    # Colab에서 결과 영상을 확인하기 위해 mp4 변환
    !ffmpeg -y -loglevel error -i face_detect_out.avi -vcodec libx264 face_detect_out.mp4
    if os.path.exists('face_detect_out.mp4'):
        display(Video('face_detect_out.mp4', embed=True))
    else:
        print("MP4 conversion failed.")


# Connected Component Analysis (연결 요소 분석)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# 빈 검정 이미지 생성
img = np.zeros((400, 400), dtype=np.uint8)
# 다양한 도형 추가
cv2.rectangle(img, (50, 50), (150, 150), 255, -1)  # 사각형
True
cv2.circle(img, (300, 100), 50, 255, -1)           # 원
cv2.ellipse(img, (100, 300), (50, 25), 0, 0, 360, 255, -1) # 타원
cv2.rectangle(img, (250, 250), (350, 350), 255, -1) # 추가 사각형
# 결과 이미지 표시
plt.imshow(img, cmap='gray')
plt.title('Shapes for Connected Component Analysis')
plt.axis('off')
plt.show()
# 이미지 저장하기
cv2.imwrite('shapes.png', img)


In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
# 간단히 shapes.png 같은 이미지 다운로드 (혹은 직접 업로드)
#!wget -q https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEinbMTeQJBoDNjR0N2weYbq9_2KmB8f922nqDeCskUls7Dm
img_path = '귀여움.jpg'
if not os.path.exists(img_path):
    print(f"{img_path} not found. Please upload or replace path.")
else:
    img = cv.imread(img_path, cv.IMREAD_COLOR)
    if img is None:
        print(f"Could not read {img_path}. Please check the file.")
    else:
        # 그레이스케일
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        # 간단히 이진화 (Threshold)
        # 오츠 자동 임계값 등 사용 가능
        ret, bin_img = cv.threshold(gray, 127, 255, cv.THRESH_BINARY_INV)
        # 연결 요소 분석
        num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(bin_img, connectivity=4)
        print(f"Detected labels (including background): {num_labels}")
        # stats.shape = (num_labels, 5)  -> [ [x, y, w, h, area], ... ]
        # centroids.shape = (num_labels, 2)
        # 시각화 위해 레이블 별로 임의 색상 입힐 수 있음
        # label 0은 배경
        label_colors = np.zeros((num_labels, 3), dtype=np.uint8)
        for i in range(1, num_labels):
            # 배경 제외, 임의 색
            label_colors[i] = np.random.randint(0, 255, size=3)
        # label map -> color
        out_img = np.zeros((bin_img.shape[0], bin_img.shape[1], 3), dtype=np.uint8)
        for r in range(bin_img.shape[0]):
            for c in range(bin_img.shape[1]):
                lb = labels[r, c]
                out_img[r, c] = label_colors[lb]
        # (옵션) 필터링 예시: area가 너무 작은 blob은 제거
        min_area = 60
        # 결과 이미지를 복사
        filtered_img = out_img.copy()
        for i in range(1, num_labels):
            area_i = stats[i, cv.CC_STAT_AREA]
            if area_i < min_area:
                # 해당 레이블을 배경색(0,0,0)으로 만듦
                # 혹은 실제 bin_img등을 바꿀 수도 있음
                filtered_img[labels==i] = (0,0,0)
        # 시각화
        plt.figure(figsize=(12,6))
        plt.subplot(1,3,1)
        plt.title('Original')
        plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        plt.axis('off')

        plt.subplot(1,3,2)
        plt.title('Connected Components')
        plt.imshow(out_img[..., ::-1])  # BGR -> RGB
        plt.axis('off')
        plt.subplot(1,3,3)
        plt.title('Filtered (area>=50)')
        plt.imshow(filtered_img[..., ::-1])
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        print("\nStats Info:")
        for i in range(num_labels):
            x, y, w, h, area = stats[i]
            cx, cy = centroids[i]
            print(f"Label={i}, BBox=({x},{y},{w},{h}), Area={area}, Centroid=({cx:.1f},{cy:.1f})")

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# 이전 단계에서 만든 이미지 로드
img = cv2.imread('귀여움.jpg', cv2.IMREAD_GRAYSCALE)

if img is None:
    print("Could not read 귀여움.jpg. Please check the file and ensure it's uploaded.")
else:
    # 연결 요소 분석 수행
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(img)
    print("연결된 구성 요소(Connected Components)의 개수:", num_labels - 1)  # 배경 제외
    # 각 구성 요소의 정보 출력
    for i in range(1, num_labels):  # 배경(0번)을 제외하고 시작
        print(f"Component {i}:")
        print(f" - 면적(Area): {stats[i, cv2.CC_STAT_AREA]}")
        print(f" - Bounding Box: (x: {stats[i, cv2.CC_STAT_LEFT]}, "
              f"y: {stats[i, cv2.CC_STAT_TOP]}, "
              f"w: {stats[i, cv2.CC_STAT_WIDTH]}, "
              f"h: {stats[i, cv2.CC_STAT_HEIGHT]})")
        print(f" - 중심점(Centroid): {centroids[i]}")
    # 시각화
    plt.figure(figsize=(8, 8))
    plt.imshow(labels, cmap='nipy_spectral')
    plt.title('Connected Components')
    plt.axis('off')
    plt.colorbar()
    plt.show()

In [ ]:
# prompt: 다른 바이너리 이미지(예: 영상에 임계값을 적용한 결과)로 연결 요소 분석을 시도해보세요.

import matplotlib.pyplot as plt
import numpy as np
import cv2 # cv2를 다시 임포트합니다.

# '귀여움.jpg' 이미지 로드
img_path = '귀여움.jpg'
img = cv2.imread(img_path, cv2.IMREAD_COLOR)

if img is None:
    print(f"Could not read {img_path}. Please check the file and ensure it's uploaded.")
else:
    # 그레이스케일 변환
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 이진화 (Threshold) - 필요에 따라 임계값 또는 임계값 방법을 변경할 수 있습니다.
    # 여기서는 간단한 고정 임계값(127)을 사용하고, 검은색 객체를 흰색 배경에서 분리하기 위해 THRESH_BINARY_INV를 사용합니다.
    ret, binary_img = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY_INV)

    # 이진 이미지 시각화
    plt.figure(figsize=(6,6))
    plt.imshow(binary_img, cmap='gray')
    plt.title(f'Binary Image from {img_path}')
    plt.axis('off')
    plt.show()

    # 연결 요소 분석
    num_labels_new, labels_new, stats_new, centroids_new = cv2.connectedComponentsWithStats(binary_img, connectivity=8) # connectivity=8 사용

    print(f"\n'{img_path}' 이미지에서 검출된 레이블 (배경 포함): {num_labels_new}")

    # 배경을 제외한 연결 요소 개수
    print(f"배경 제외 연결된 구성 요소(Connected Components)의 개수: {num_labels_new - 1}")

    # 각 구성 요소 정보 출력 및 시각화 준비
    # 원본 컬러 이미지를 복사하여 텍스트와 BBox를 그립니다.
    output_img_new_colored = img.copy()

    # 배경 제외하고 임의 색상 생성 (컴포넌트 영역 색칠용)
    label_colors_new = np.random.randint(0, 255, size=(num_labels_new, 3), dtype=np.uint8)
    label_colors_new[0] = [0, 0, 0] # 배경은 검은색

    # 연결 요소 영역을 색상으로 표시할 이미지 (옵션)
    colored_components_img = np.zeros_like(output_img_new_colored)


    print("\nStats Info (using '귀여움.jpg'):")
    for i in range(1, num_labels_new): # 배경 (label 0) 제외
        x, y, w, h, area = stats_new[i]
        cx, cy = centroids_new[i]

        print(f"Label={i}, BBox=({x},{y},{w},{h}), Area={area}, Centroid=({cx:.1f},{cy:.1f})")

        # 결과 이미지에 Bounding box 그리기
        cv2.rectangle(output_img_new_colored, (x, y), (x + w, y + h), (0, 255, 0), 2) # 초록색 BBox

        # 결과 이미지에 중심점 그리기
        cv2.circle(output_img_new_colored, (int(cx), int(cy)), 5, (255, 0, 0), -1) # 파란색 중심점

        # (옵션) 연결 요소 영역을 색상으로 채우기
        colored_components_img[labels_new == i] = label_colors_new[i]


    # 결과 시각화 (원본 + BBox/Centroid)
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.title(f'Original Image with Components ({img_path})')
    plt.imshow(output_img_new_colored[..., ::-1]) # BGR to RGB
    plt.axis('off')

    # (옵션) 연결 요소 색칠 이미지 시각화
    plt.subplot(1, 2, 2)
    plt.title('Connected Components (Colored)')
    plt.imshow(colored_components_img[..., ::-1]) # BGR to RGB
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # 레이블 맵만 별도로 시각화 (색상맵 사용)
    plt.figure(figsize=(6, 6))
    plt.imshow(labels_new, cmap='nipy_spectral') # 레이블 값 자체를 색상맵으로 표현
    plt.title(f'Connected Components Label Map ({img_path})')
    plt.axis('off')
    plt.colorbar(label='Component Label')
    plt.show()


    # (옵션) 필터링 예시: 특정 면적 이상인 컴포넌트만 표시
    min_area_filter = 100 # 예시 필터링 기준
    filtered_output_img = np.zeros_like(output_img_new_colored) # 원본 컬러 이미지 크기로 초기화
    filtered_labels_count = 0

    print(f"\nFiltering with conditions: Area >= {min_area_filter}")

    for i in range(1, num_labels_new):
        x, y, w, h, area = stats_new[i]
        cx, cy = centroids_new[i]

        if area >= min_area_filter:
            # 필터링된 컴포넌트의 Bounding Box와 중심점을 결과 이미지에 그립니다.
            cv2.rectangle(filtered_output_img, (x, y), (x + w, y + h), (0, 255, 0), 2) # 초록색 BBox
            cv2.circle(filtered_output_img, (int(cx), int(cy)), 5, (255, 0, 0), -1) # 파란색 중심점

            filtered_labels_count += 1
            # 필터링된 컴포넌트 정보 다시 출력
            print(f"Filtered Component {i}: BBox=({x},{y},{w},{h}), Area={area}, Centroid=({cx:.1f},{cy:.1f})")


    if filtered_labels_count > 0:
        plt.figure(figsize=(6, 6))
        plt.title(f'Filtered Components (Area >= {min_area_filter}) on {img_path}')
        plt.imshow(filtered_output_img[..., ::-1]) # BGR to RGB
        plt.axis('off')
        plt.show()
        print(f"필터링 후 남은 연결된 구성 요소 개수: {filtered_labels_count}")
    else:
        print(f"면적이 {min_area_filter} 이상인 연결된 구성 요소가 없습니다.")

In [ ]:
# prompt: 연결성: connectivity=8 로 바꾸고 결과가 달라지는지 확인해보세요.

import matplotlib.pyplot as plt
# 이전 단계에서 만든 이미지 로드
img = cv2.imread('귀여움.jpg', cv2.IMREAD_GRAYSCALE)

if img is None:
    print("Could not read 귀여움.jpg. Please check the file and ensure it's uploaded.")
else:
    # 연결 요소 분석 수행 (connectivity=8 사용)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=8)

    print("연결된 구성 요소(Connected Components)의 개수 (connectivity=8):", num_labels - 1)  # 배경 제외

    # 각 구성 요소의 정보 출력
    print("\nStats Info (connectivity=8):")
    for i in range(1, num_labels):  # 배경(0번)을 제외하고 시작
        print(f"Component {i}:")
        print(f" - 면적(Area): {stats[i, cv2.CC_STAT_AREA]}")
        print(f" - Bounding Box: (x: {stats[i, cv2.CC_STAT_LEFT]}, "
              f"y: {stats[i, cv2.CC_STAT_TOP]}, "
              f"w: {stats[i, cv2.CC_STAT_WIDTH]}, "
              f"h: {stats[i, cv2.CC_STAT_HEIGHT]})")
        print(f" - 중심점(Centroid): {centroids[i]}")

    # 시각화
    plt.figure(figsize=(8, 8))
    plt.imshow(labels, cmap='nipy_spectral')
    plt.title('Connected Components (connectivity=8)')
    plt.axis('off')
    plt.colorbar()
    plt.show()

    # 이전 코드와의 결과 비교
    # connectivity=4일 때와 connectivity=8일 때의 연결 요소 개수 및 stats 값을 비교하여 결과가 달라지는지 확인합니다.
    # 일반적으로 8-connectivity는 대각선으로 인접한 픽셀도 같은 연결 요소로 간주하므로, 4-connectivity보다 연결 요소의 개수가 적거나 같게 나옵니다.
    # 현재 'shapes.png' 이미지의 경우, 사각형, 원, 타원 등은 4-connectivity와 8-connectivity 모두 동일하게 분리된 객체로 인식될 가능성이 높습니다.
    # 하지만 만약 대각선으로만 연결된 픽셀들이 있다면, connectivity=8일 때 하나의 연결 요소로 합쳐지게 되어 connectivity=4일 때와 결과가 달라질 수 있습니다.

    print("\nNote: Compare the number of components and stats with the connectivity=4 result above to see the difference.")
    print("For the '귀여움.jpg' image, the difference will depend on the nature of the shapes and connections within the image.")
    print("Differences are more apparent in images with diagonally connected pixels or noisy boundaries.")

In [ ]:
# prompt: 필터링: stats 의 다양한 정보(예: 너비, 높이, aspect ratio 등)를 사용해 조건부 필터링을 적용해보세요.

import matplotlib.pyplot as plt
import numpy as np
# 다양한 stats 정보를 사용하여 조건부 필터링 적용 예시
# stats 배열은 각 컴포넌트의 [x, y, width, height, area] 정보를 담고 있습니다.

print("\nApplying various filtering conditions based on stats:")

# 이전 단계에서 사용한 이미지와 분석 결과를 로드합니다.
# 여기서는 귀여움.jpg 이미지와 그 분석 결과 (num_labels, labels, stats, centroids)를 사용합니다.
img = cv2.imread('귀여움.jpg', cv2.IMREAD_GRAYSCALE)

if img is None:
    print("Could not read 귀여움.jpg. Please check the file and ensure it's uploaded.")
else:
    # 연결 요소 분석 수행 (이전 단계에서 수행한 결과가 없으면 다시 수행)
    num_labels_new, labels_new, stats_new, centroids_new = cv2.connectedComponentsWithStats(img, connectivity=8) # connectivity=8 사용

    # 필터링 조건을 정의합니다.
    min_area_filter = 500  # 최소 면적
    min_width_filter = 50  # 최소 너비
    max_height_filter = 150 # 최대 높이
    min_aspect_ratio = 0.8 # 최소 가로/세로 비율 (정사각형에 가까운 형태)
    max_aspect_ratio = 1.2 # 최대 가로/세로 비율

    # 텍스트를 그릴 이미지 준비 (필터링 결과를 보여줄 이미지)
    # 원본 바이너리 이미지를 3채널 컬러로 변환하여 사용합니다.
    filtered_output_img_complex = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    filtered_labels_count_complex = 0

    print(f"\nFiltering with conditions: Area >= {min_area_filter}, Width >= {min_width_filter}, Height <= {max_height_filter}, Aspect Ratio between {min_aspect_ratio:.1f} and {max_aspect_ratio:.1f}")

    for i in range(1, num_labels_new): # 배경 (label 0) 제외
        x, y, w, h, area = stats_new[i]
        cx, cy = centroids_new[i]

        # Aspect Ratio 계산 (Divide by zero 방지)
        aspect_ratio = w / h if h != 0 else 0

        # 필터링 조건 확인
        is_large_enough_area = area >= min_area_filter
        is_wide_enough = w >= min_width_filter
        is_short_enough = h <= max_height_filter
        is_aspect_ratio_ok = (aspect_ratio >= min_aspect_ratio) and (aspect_ratio <= max_aspect_ratio) if h != 0 else False # 높이가 0인 경우 비율 체크 제외

        # 모든 조건을 만족하는 경우 필터링된 이미지에 추가
        if is_large_enough_area and is_wide_enough and is_short_enough and is_aspect_ratio_ok:
            # 해당 컴포넌트를 원본 이미지에 빨간색으로 표시 (예시)
            filtered_output_img_complex[labels_new == i] = (0, 0, 255) # BGR: Red
            filtered_labels_count_complex += 1
            # 필터링된 컴포넌트 정보 다시 출력
            print(f"Filtered Component {i} (Complex Filter): BBox=({x},{y},{w},{h}), Area={area}, Aspect Ratio={aspect_ratio:.2f}, Centroid=({cx:.1f},{cy:.1f})")

    if filtered_labels_count_complex > 0:
        plt.figure(figsize=(6, 6))
        plt.title('Filtered Components (Complex Conditions)')
        plt.imshow(filtered_output_img_complex[..., ::-1]) # BGR to RGB
        plt.axis('off')
        plt.show()
        print(f"복합 필터링 후 남은 연결된 구성 요소 개수: {filtered_labels_count_complex}")
    else:
        print("복합 필터링 조건을 만족하는 연결된 구성 요소가 없습니다.")

    # 다른 필터링 예시: 특정 y좌표 범위에 있는 컴포넌트만 필터링
    min_y_filter = 200
    max_y_filter = 400 # 이미지 하단에 있는 컴포넌트

    filtered_output_img_y = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) # 새로운 필터링 결과 이미지 초기화
    filtered_labels_count_y = 0

    print(f"\nFiltering with conditions: Y coordinate (Top) between {min_y_filter} and {max_y_filter}")

    for i in range(1, num_labels_new):
        x, y, w, h, area = stats_new[i]
        cx, cy = centroids_new[i]

        # Y 좌표 조건 확인 (컴포넌트의 상단 y좌표 사용)
        is_in_y_range = (y >= min_y_filter) and (y <= max_y_filter)

        if is_in_y_range:
             filtered_output_img_y[labels_new == i] = (0, 255, 0) # BGR: Green
             filtered_labels_count_y += 1
             print(f"Filtered Component {i} (Y-Range Filter): BBox=({x},{y},{w},{h}), Area={area}, Centroid=({cx:.1f},{cy:.1f})")

    if filtered_labels_count_y > 0:
        plt.figure(figsize=(6, 6))
        plt.title(f'Filtered Components (Y between {min_y_filter} and {max_y_filter})')
        plt.imshow(filtered_output_img_y[..., ::-1]) # BGR to RGB
        plt.axis('off')
        plt.show()
        print(f"Y좌표 범위 필터링 후 남은 연결된 구성 요소 개수: {filtered_labels_count_y}")
    else:
        print(f"Y좌표 범위 ({min_y_filter}~{max_y_filter}) 내에 있는 연결된 구성 요소가 없습니다.")

In [ ]:
# prompt: 4. 중심 좌표를 이용해, 각 레이블 위치에 텍스트 표시를 해보는 것도 좋습니다 ( cv.putText ).

import matplotlib.pyplot as plt
import numpy as np
import cv2 # cv2를 다시 임포트합니다.
# 중심 좌표를 이용해, 각 레이블 위치에 텍스트 표시를 해보는 것도 좋습니다 ( cv.putText ).

# 이전 단계에서 사용한 이미지와 분석 결과를 사용합니다.
# 여기서는 귀여움.jpg 이미지와 그 분석 결과 (num_labels, labels, stats, centroids)를 사용합니다.
img_path = '귀여움.jpg'
img = cv2.imread(img_path, cv2.IMREAD_COLOR) # 컬러 이미지로 로드

if img is None:
    print(f"Could not read {img_path}. Please check the file and ensure it's uploaded.")
else:
    # 연결 요소 분석을 위해 그레이스케일 및 이진화가 필요할 수 있습니다.
    # 이 예제에서는 원본 컬러 이미지에 결과를 그릴 것이므로, 연결 요소 분석은 그레이스케일/이진화된 이미지에서 수행합니다.
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, bin_img = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV) # 이진화 (필요에 따라 임계값 조절)


    # 연결 요소 분석 수행 (이전 단계에서 수행한 결과가 없으면 다시 수행)
    num_labels_new, labels_new, stats_new, centroids_new = cv2.connectedComponentsWithStats(bin_img, connectivity=8) # 이진 이미지에 대해 분석 수행


    # 텍스트를 그릴 이미지 준비 (원본 컬러 이미지에 그립니다.)
    img_with_text = img.copy()


    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    font_color = (0, 0, 255)  # 파란색 (BGR 순서)
    thickness = 1

    print("\nDisplaying centroid coordinates and label numbers on components:")

    # 배경 (label 0)을 제외하고 각 연결 요소에 텍스트를 표시합니다.
    for i in range(1, num_labels_new):
        x, y, w, h, area = stats_new[i]
        cx, cy = centroids_new[i]


        # 중심 좌표 근처에 텍스트를 그립니다.
        # putText 위치는 텍스트의 좌측 하단 기준입니다.
        # 중심점 (cx, cy)에 정확히 찍기보다는, 약간 조정하여 텍스트가 컴포넌트 내부에 위치하도록 할 수 있습니다.
        # 여기서는 간단히 Bounding Box의 좌측 상단 근처에 레이블 ID를 표시합니다.
        text_str = f"{i}" #f"({cx:.0f},{cy:.0f})" # 레이블 번호나 좌표 등을 표시할 수 있습니다.
        # 텍스트 위치를 BBox의 좌측 상단 근처로 조정
        text_position = (x, y - 5) # BBox 상단에서 약간 위로 이동


        # 텍스트가 너무 작거나 컴포넌트가 너무 작으면 그리지 않을 수도 있습니다.
        if w > 5 and h > 5: # 간단한 크기 필터링 (너무 작은 노이즈 제외)
           # 텍스트가 이미지 경계를 벗어나지 않도록 위치 조정
           text_position = (max(0, text_position[0]), max(10, text_position[1])) # x, y 최소값 제한

           # 이미지에 텍스트 그리기
           cv2.putText(img_with_text, text_str, text_position, font, font_scale, font_color, thickness, cv2.LINE_AA)

           # (옵션) Bounding Box 그리기
           cv2.rectangle(img_with_text, (x, y), (x+w, y+h), (255, 0, 0), 1) # 빨간색 BBox

           # (옵션) 중심점 표시
           #cv2.circle(img_with_text, (int(cx), int(cy)), 3, (0, 255, 0), -1) # 초록색 중심점

    # 결과 이미지 시각화
    plt.figure(figsize=(8, 8))
    plt.imshow(img_with_text[..., ::-1])  # BGR to RGB
    plt.title(f'Components with Labels on {img_path}')
    plt.axis('off')
    plt.show()

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os

img_path = '귀여움.jpg'

if not os.path.exists(img_path):
    print(f"{img_path} not found. Please upload the image.")
else:
    img = cv.imread(img_path, cv.IMREAD_COLOR)

    if img is None:
        print(f"Could not read {img_path}. Please check the file.")
    else:
        # 그레이스케일 변환
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # 이진화 (Threshold) - 필요에 따라 임계값 또는 임계값 방법을 변경할 수 있습니다.
        # THRESH_BINARY_INV는 검은색 배경에 흰색 객체를 만들 때 유용합니다.
        # 귀여움.jpg 이미지 특성에 맞게 임계값(예: 127)을 조절하거나 다른 방법(THRESH_BINARY, THRESH_OTSU 등)을 시도해 볼 수 있습니다.
        ret, bin_img = cv.threshold(gray, 127, 255, cv.THRESH_BINARY_INV)

        # 연결 요소 분석
        # connectivity=8 사용 (대각선 포함)
        num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(bin_img, connectivity=8)

        print(f"Detected labels (including background): {num_labels}")

        # 시각화 위해 레이블 별로 임의 색상 입힐 수 있음
        # label 0은 배경
        label_colors = np.zeros((num_labels, 3), dtype=np.uint8)
        # 배경 제외, 임의 색상 생성
        for i in range(1, num_labels):
            label_colors[i] = np.random.randint(0, 255, size=3)

        # label map -> color image
        out_img_colored = np.zeros((bin_img.shape[0], bin_img.shape[1], 3), dtype=np.uint8)
        for r in range(bin_img.shape[0]):
            for c in range(bin_img.shape[1]):
                lb = labels[r, c]
                out_img_colored[r, c] = label_colors[lb]

        # 원본 이미지 위에 경계 상자 그리기 (옵션)
        img_with_bbox = img.copy()
        # 중심점 표시 (옵션)
        # img_with_centroids = img.copy()

        print("\nStats Info:")
        for i in range(1, num_labels): # 배경 (label 0) 제외
            x, y, w, h, area = stats[i]
            cx, cy = centroids[i]

            print(f"Label={i}, BBox=({x},{y},{w},{h}), Area={area}, Centroid=({cx:.1f},{cy:.1f})")

            # 원본 이미지 위에 경계 상자 그리기
            cv.rectangle(img_with_bbox, (x, y), (x+w, y+h), (0, 255, 0), 2) # 초록색 경계 상자

            # 중심점 표시 (옵션)
            # cv.circle(img_with_centroids, (int(cx), int(cy)), 5, (255, 0, 0), -1) # 파란색 중심점

            # (옵션) 컴포넌트 레이블 텍스트 추가
            # text_str = f"{i}"
            # text_position = (x, y - 5)
            # cv.putText(img_with_bbox, text_str, text_position, cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv.LINE_AA)


        # 시각화
        plt.figure(figsize=(18, 6)) # 전체 Figure 크기 조정

        plt.subplot(1, 3, 1)
        plt.title('Original Image')
        plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title('Connected Components (Colored)')
        # OpenCV는 BGR 순서, Matplotlib은 RGB 순서이므로 색상 채널 순서 변경
        plt.imshow(out_img_colored[..., ::-1])
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title('Original with Bounding Boxes')
        # OpenCV는 BGR 순서, Matplotlib은 RGB 순서이므로 색상 채널 순서 변경
        plt.imshow(img_with_bbox[..., ::-1])
        plt.axis('off')

        plt.tight_layout()
        plt.show()

        # (옵션) 이진 이미지 시각화 (디버깅용)
        # plt.figure(figsize=(6,6))
        # plt.imshow(bin_img, cmap='gray')
        # plt.title('Binary Image')
        # plt.axis('off')
        # plt.show()

        # (옵션) 레이블 맵만 시각화 (컴포넌트 구별 확인용)
        # plt.figure(figsize=(6,6))
        # plt.imshow(labels, cmap='nipy_spectral')
        # plt.title('Label Map')
        # plt.axis('off')
        # plt.colorbar()
        # plt.show()